<a href="https://colab.research.google.com/github/Map987/BilibiliSummary/blob/main/colab%E4%B8%8B%E8%BD%BD%E6%96%B9%E6%A1%88_share1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown #### 说明：左边查看目录，文件列表，右键可复制文件（夹）路径

In [ ]:
#@title # 谷歌云盘挂载到Colab环境（必选）
from google.colab import drive
drive.mount('/content/drive')
!apt-get -y install -qq aria2

#（谷歌和OneDrive）下载

In [ ]:
#@title # （谷歌云盘）文件夹下载、单文件下载
import os
folder = "/content/" #@param {type:"string"}
folder = os.path.join(folder, "")
url = "https://drive.google.com/drive/folders/xxx" #@param {type:"string"}

if "/folders/" in url:
  !gdown --fuzzy {url} -O {folder} --folder
else:
  !gdown --fuzzy {url} -O {folder}

In [ ]:
#@title # （OneDrive）文件夹下载，单文件下载
import os
import re
import requests
import urllib.parse
!apt-get -y install -qq aria2
def onelink1(string):
    !echo -n "{string}"|base64|sed "s/=$//;s/\//\_/g;s/\+/\-/g;s/^/https:\/\/api\.onedrive\.com\/v1\.0\/shares\/u\!/;s/$/\/root\/content/";
# 将分享链接转换为直链下载链接
def onelink(string):
    cmd = f"echo -n '{string}' | base64 | sed 's/=$//;s/\//_/g;s/+/\\-/g;s/^/https:\\/\\/api.onedrive.com\\/v1.0\\/shares\\/u!/;s/$/\\/root\\/content/'"
    return [os.popen(cmd).read().strip()]
def get_download_url(url):
    response = requests.get(url, allow_redirects=False)
    redirect_url = response.headers.get('location')
    # 从原链接中提取出第一部分、第二部分和第三部分
    parts = redirect_url.split("/")
    first_part = "/".join(parts[:3])
    second_part = "/".join(parts[3:7])
    third_part = "download.aspx?SourceUrl=" + "/".join(parts[7].split("=")[1].split("&")[:-1])
    # 组合成目标链接
    target_link = first_part + "/" + second_part + "/" + third_part
    return target_link
def get_download_url_sp(url,cookie):
    headers = {
        'Cookie': cookie
    }
    response = requests.get(url, allow_redirects=False,headers=headers)
    redirect_url = response.headers.get('location')
    decoded_url = urllib.parse.unquote(redirect_url)
    url = decoded_url
    base_path = '/'.join(url.split('/')[:5]) + '/'
    base_path2 = '/'.join(url.split('/')[:7]) + '/'
    # 从原链接中获取文件夹路径
    folder_path = url.split("id=")[1].split("&")[0]
    site_path = '/'.join((url.split("id=")[1]).split('/')[:4])
    root_folder_param = "&RootFolder=" + folder_path
    get_list_using_path_param = f"@a1='{site_path}'"
    try_new_experience_single_param = "TryNewExperienceSingle=TRUE"
    post_url = base_path+"_api/web/GetListUsingPath(DecodedUrl=@a1)/RenderListDataAsStream?" + get_list_using_path_param + root_folder_param + "&" + try_new_experience_single_param
    response = requests.post(post_url,headers=headers)
    response_dict = response.json()
    download_urls = []
    file_names = []
    for row in response_dict['Row']:
        file_path = row['FileRef']
        file_name = row['FileLeafRef']
        download_url = base_path2 + "download.aspx?SourceUrl=" + file_path
        download_urls.append(download_url)

    print(download_urls)
    with open('urls.txt', 'w') as f:
        f.writelines([url + '\n' for url in download_urls])
    !aria2c --header "Cookie: {cookie}" --summary-interval 10 --console-log-level=notice -c -x 16 -s 16 -k 1M  -d {folder} -i urls.txt


#@markdown 保存路径
folder = "/content/sample_sd" #@param {type:"string"}
folder = os.path.join(folder, "")
link = "" #@param {type:"string"}


if "sharepoint" in link:
  print("This is a Onedrive sharepoint link.")
  cookie = os.popen(f'curl -L {link} -A "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36" -s -I | grep set-cookie | grep FedAuth | sed "s/set-cookie: //" | sed "s/;.*//"').read().strip()
  if ":f:" in link:
    print("This is a folder link.")
    get_download_url_sp(link,cookie)
  elif ":u:" in link:
    print("This is a file link.")
    direct_link = get_download_url(link)
    print(direct_link)
    !aria2c --header "Cookie: {cookie}" --summary-interval 10 --console-log-level=notice -c -x 16 -s 16 -k 1M "{direct_link}" -d {folder}
  else:
    print("Invalid link.")
else:
    print("This is a Onedrive personal link.")
    if "/u/" in link:
        print("This is a file link.")
        url = onelink(link)[0]
        print(url)
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {folder}
    elif "/f/" in link:

        print("This is a folder link.")
        decoded_url = urllib.parse.unquote(link)
        print(decoded_url)
        # 将要转换的链接
        decoded_url = "https://onedrive.live.com/redir?resid=EB170A97A2E583E3!24655&authkey=!AE8inL5QvAG_sW4&ithint=folder"
        # 从链接中提取资源ID和项目ID
        resource_id = re.search(r'resid=(.*?)!', decoded_url).group(1)
        item_id = re.search(r'!(.*?)&', decoded_url).group(1)
        # 构建转换后的链接
        api_url = f"https://api.onedrive.com/v1.0/drives/{resource_id}/items/{resource_id}!{item_id}/children?&authKey=!AE8inL5QvAG_sW4"
        print(api_url)
        # 发送API请求，获取响应数据
        response = requests.get(api_url)
        response_dict = response.json()
        # 获取所有文件的@content.downloadUrl
        download_urls = []
        for item in response_dict['value']:
            download_url = item['@content.downloadUrl']
            download_urls.append(download_url)

        print(download_urls)
        with open('urls.txt', 'w') as f:
          f.writelines([url + '\n' for url in download_urls])
        !aria2c --summary-interval 10 --console-log-level=notice -c -x 16 -s 16 -k 1M  -d {folder} -i urls.txt
    else:
        print("Invalid link.")

# (百度网盘）下载或上传

In [ ]:
#@title 1.安装依赖
!pip3 install Cython
!pip3 install BaiduPCS-Py
!pip3 install aget

In [ ]:
#@title 2.链接百度网盘
from baidupcs_py.baidupcs import BaiduPCSApi
from http.cookies import SimpleCookie
#@markdown 打开百度网盘，F12查看网络请求，随便打开一个文件夹，复制list?xxx请求下面的cookie,bduss在cookies里面有
cookies = ""  #@param {type:"string"}
bduss = ""  #@param {type:"string"}

cookies_dict = {k.strip(): v.value for k, v in SimpleCookie(cookies).items()}

print(cookies_dict)
#@markdown 这里随便在控制台输个东西回车

api = BaiduPCSApi(bduss=bduss, cookies=cookies_dict)
!BaiduPCS-Py useradd --cookies "{cookies}" --bduss "{bduss}"
!BaiduPCS-Py who

In [ ]:
#@title 3.下载
import time
import random
import string
import subprocess
import os
#@markdown 多线程批量后台下载，可以自己看下载文件夹进度
#@markdown 填百度网盘文件夹路径
folder_path = "/apps" #@param {type:"string"}
# pass = "j3vk" #@param {type:"string"}
save_path = "/content" #@param {type:"string"}
!BaiduPCS-Py ls -n -A "{folder_path}"
# !BaiduPCS-Py download  {folder_path} -o {save_path}
# !BaiduPCS-Py download -d aget_py {folder_path} -o {save_path}
output = get_ipython().getoutput(f'BaiduPCS-Py ls -n -A "{folder_path}"')

file_paths = [line[13:].strip() for line in output[3:]]
print(file_paths)
for file_path in file_paths:
    with open("download_log1.txt", "a") as log_file:
      subprocess.Popen(['BaiduPCS-Py', 'download', '-d', 'aget_py', file_path, '-o', save_path], stdout=log_file, stderr=log_file)

In [ ]:
#@title 4.上传
file_path = "" #@param {type:"string"}
file_savepath = "apps" #@param {type:"string"}
#@markdown 线程数，玄学参数
max_workers = 4 #@param {type:"slider", min:1, max:64, step:1}
#@markdown 单文件上传或多文件上传
upload_type = "One" #@param ["One", "Many"]
!BaiduPCS-Py upload --upload-type One --max-workers {max_workers}  {file_path} {file_savepath}

# （rclone） 挂载多种云盘，互相同步
Modified from https://colab.research.google.com/github/shirooo39/MiXLab/blob/master/MiXLab.ipynb#scrollTo=3GKtYuBbUP-c

In [ ]:
#@title 1.安装rclone
import os
# !curl https://rclone.org/install.sh | sudo bash
# !sudo apt-get -y install fuse3
!mkdir -p "/root/.config/rclone"


In [ ]:
#@title 2.配置rclone
#@markdown 建议在本机配置后再上传配置文件，<a href="https://rclone.org/downloads/">下载rclone安装包</a>解压，在目录下敲命令./rclone config，<a href="https://hechuan.me/rclone-google-drive">配置过程</a>，配置文件保存在C:\Users\xxx\AppData\Roaming\rclone）
!rclone config

In [ ]:
#@title （可选）下载/上传配置文件
from google.colab import files
is_upload = True #@param {type:"boolean"}
if is_upload:
  !rm /content/rclone.conf
  uploaded = files.upload()
  filepath = list(uploaded.keys())[0]
  !mkdir -p "/root/.config/rclone"
  !cp -f "/content/{filepath}" /root/.config/rclone
else:
  files.download('/root/.config/rclone/rclone.conf')

In [ ]:
#@title （可选）挂载到colab
# ============================= FORM ============================= #
Cache_Directory = "DISK" #@param ["RAM", "DISK"]
# ================================================================ #

import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re

if not os.path.exists("/root/.ipython/mixlab.py"):
  from shlex import split as _spl
  from subprocess import run

  shellCmd = "wget -qq https://shirooo39.github.io/MiXLab/resources/mixlab.py \
                  -O /root/.ipython/mixlab.py"
  run(_spl(shellCmd))

from mixlab import (
    runSh,
    prepareSession,
    rcloneConfigurationPath,
)

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html

def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#4f8bd6;margin-top:0px;">
                  Available drive to mount/unmount:</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )

  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))

prepareSession()
content = open(f"{rcloneConfigurationPath}/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)

if Cache_Directory == 'RAM':
  cache_path = '/dev/shm'
elif Cache_Directory == 'DISK':
  os.makedirs('/tmp', exist_ok=True)
  cache_path = '/tmp'

def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    cmd = rf"rclone mount {mountNam.value}: {mPoint}" \
      rf" --config {rcloneConfigurationPath}/rclone.conf" \
      ' --user-agent "Mozilla"' \
      ' --buffer-size 256M' \
      ' --transfers 10' \
      ' --vfs-cache-mode full' \
      ' --vfs-cache-max-age 0h0m1s' \
      ' --vfs-cache-poll-interval 0m1s' \
      f' --cache-dir {cache_path}' \
      ' --allow-other' \
      ' --daemon'

    if runSh(cmd, shell=True) == 0:
      print(f"The drive have been successfully mounted! - \t{mPoint}")
    else:
      print(f"Failed to mount the drive! - \t{mPoint}")

def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"The drive have been successfully unmounted! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)

ShowAC()

In [ ]:
#@title 3.启动rclone
#@markdown 具体参数说明见官方文档或 <a href="https://ld246.com/article/1600853705300#%E5%85%B3%E4%BA%8E-rclone">这个中文文档</a>
#@markdown ---
#@markdown ---
# ============================= FORM ============================= #
# @markdown 同步模式：Sync可能会删除文件，使用前选中Dry_Run(运行测试)
Mode = "Copy" #@param ["Copy", "Move", "Sync", "Checker", "Deduplicate", "Remove Empty Directories", "Empty Trash"]
#@markdown 配置时输入的名称:路径，如 onedrive1:xxx/xxx
Source = "od:apk" #@param {type:"string"}
Destination = "gd:test" #@param {type:"string"}
#@markdown 仅运行测试，不修改文件
Dry_Run = False #@param {type:"boolean"}
#@markdown ---
#@markdown <center><h3><font color="#3399ff"><b>⚙️ Global Configuration ⚙️</b></font></h3></center>
Extra_Arguments = "" #@param {type:"string"}
Compare = "Size & Mod-Time" #@param ["Size & Mod-Time", "Size & Checksum", "Only Mod-Time", "Only Size", "Only Checksum"]
#@markdown 最大比较文件的数量和同步传输的文件数量，调大可能会被限制
Checkers = 12 #@param {type:"slider", min:1, max:40, step:1}
Transfers = 12 #@param {type:"slider", min:1, max:20, step:1}

Do_not_cross_filesystem_boundaries = False
Do_not_update_modtime_if_files_are_identical = False #@param {type:"boolean"}
Google_Drive_optimization = False #@param {type:"boolean"}
Large_amount_of_files_optimization = False #@param {type:"boolean"}
Simple_Ouput = True #@param {type:"boolean"}
Skip_all_files_that_exist = False #@param {type:"boolean"}
Skip_files_that_are_newer_on_the_destination = False #@param {type:"boolean"}
Output_Log_File = "OFF" #@param ["OFF", "NOTICE", "INFO", "ERROR", "DEBUG"]

#@markdown <br><center><h3><font color="#3399ff"><b>↪️ Sync Configuration ↩️</b></font></h3></center>
Sync_Mode = "Delete during transfer" #@param ["Delete during transfer", "Delete before transfering", "Delete after transfering"]
Track_Renames = False #@param {type:"boolean"}

#@markdown <br><center><h3><font color="#3399ff"><b>💞 Deduplicate Configuration 💞</b></font></h3></center>
Deduplicate_Mode = "Interactive" #@param ["Interactive", "Skip", "First", "Newest", "Oldest", "Largest", "Rename"]
Deduplicate_Use_Trash = True #@param {type:"boolean"}

#@markdown ---
automatically_clear_cell_output = False  # @param{type: "boolean"}
# ================================================================ #

##### Importing the needed modules
import os
import IPython
from IPython.display import HTML, clear_output


##### Variable Declaration
# Optimized for Google Colaboratory
os.environ["bufferC"] = "--buffer-size 96M"

if Compare == "Size & Checksum":
    os.environ["compareC"] = "-c"
elif Compare == "Only Mod-Time":
    os.environ["compareC"] = "--ignore-size"
elif Compare == "Only Size":
    os.environ["compareC"] = "--size-only"
elif Compare == "Only Checksum":
    os.environ["compareC"] = "-c --ignore-size"
else:
    os.environ["compareC"] = ""

os.environ["sourceC"] = Source
os.environ["destinationC"] = Destination
os.environ["transfersC"] = "--transfers "+str(Transfers)
os.environ["checkersC"] = "--checkers "+str(Checkers)

if Skip_files_that_are_newer_on_the_destination == True:
    os.environ["skipnewC"] = "-u"
else:
    os.environ["skipnewC"] = ""

if Skip_all_files_that_exist == True:
    os.environ["skipexistC"] = "--ignore-existing"
else:
    os.environ["skipexistC"] = ""

if Do_not_cross_filesystem_boundaries == True:
    os.environ["nocrossfilesystemC"] = "--one-file-system"
else:
    os.environ["nocrossfilesystemC"] = ""

if Do_not_update_modtime_if_files_are_identical == True:
    os.environ["noupdatemodtimeC"] = "--no-update-modtime"
else:
    os.environ["noupdatemodtimeC"] = ""

if Large_amount_of_files_optimization == True:
    os.environ["filesoptimizeC"] = "--fast-list"
else:
    os.environ["filesoptimizeC"] = ""

if Google_Drive_optimization == True:
    os.environ["driveoptimizeC"] = "--drive-chunk-size 32M --drive-acknowledge-abuse --drive-keep-revision-forever"
else:
    os.environ["driveoptimizeC"] = ""

if Dry_Run == True:
    os.environ["dryrunC"] = "-n"
else:
    os.environ["dryrunC"] = ""

if Output_Log_File != "OFF":
    os.environ["statsC"] = "--log-file=/root/.rclone_log/rclone_log.txt"
else:
    if Simple_Ouput == True:
        os.environ["statsC"] = "-v --stats-one-line --stats=5s"
    else:
        os.environ["statsC"] = "-v --stats=5s"

if Output_Log_File == "INFO":
    os.environ["loglevelC"] = "--log-level INFO"
elif Output_Log_File == "ERROR":
    os.environ["loglevelC"] = "--log-level ERROR"
elif Output_Log_File == "DEBUG":
    os.environ["loglevelC"] = "--log-level DEBUG"
else:
    os.environ["loglevelC"] = ""

os.environ["extraC"] = Extra_Arguments

if Sync_Mode == "Delete during transfer":
    os.environ["syncmodeC"] = "--delete-during"
elif Sync_Mode == "Delete before transfering":
    os.environ["syncmodeC"] = "--delete-before"
elif Sync_Mode == "Delete after transfering":
    os.environ["syncmodeC"] = "--delete-after"

if Track_Renames == True:
    os.environ["trackrenamesC"] = "--track-renames"
else:
    os.environ["trackrenamesC"] = ""

if Deduplicate_Mode == "Interactive":
    os.environ["deduplicateC"] = "interactive"
elif Deduplicate_Mode == "Skip":
    os.environ["deduplicateC"] = "skip"
elif Deduplicate_Mode == "First":
    os.environ["deduplicateC"] = "first"
elif Deduplicate_Mode == "Newest":
    os.environ["deduplicateC"] = "newest"
elif Deduplicate_Mode == "Oldest":
    os.environ["deduplicateC"] = "oldest"
elif Deduplicate_Mode == "Largest":
    os.environ["deduplicateC"] = "largest"
elif Deduplicate_Mode == "Rename":
    os.environ["deduplicateC"] = "rename"


if Deduplicate_Use_Trash == True:
    os.environ["deduplicatetrashC"] = ""
else:
    os.environ["deduplicatetrashC"] = "--drive-use-trash=false"


##### rclone Execution
if Output_Log_File != "OFF" and Mode != "Config":
    !mkdir -p -m 666 /root/.rclone_log/
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Logging enabled, rclone will no longer display any output on the terminal.<br>Please wait until the cell stop by itself.</h2></center><br>"))

if Mode == "Copy":
    !rclone --config=/root/.config/rclone/rclone.conf copy "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == "Move":
    !rclone --config=/root/.config/rclone/rclone.conf move "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC --delete-empty-src-dirs $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == "Sync":
    !rclone --config=/root/.config/rclone/rclone.conf sync "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC $syncmodeC $trackrenamesC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == "Checker":
    !rclone --config=/root/.config/rclone/rclone.conf check "$sourceC" "$destinationC" $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == "Deduplicate":
    !rclone --config=/root/.config/rclone/rclone.conf dedupe "$sourceC" $checkersC $statsC $loglevelC --dedupe-mode $deduplicateC $deduplicatetrashC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC
elif Mode == "Remove Empty Directories":
    !rclone --config=/root/.config/rclone/rclone.conf rmdirs "$sourceC" $statsC $loglevelC $dryrunC $extraC
elif Mode == "Empty Trash":
    !rclone --config=/root/.config/rclone/rclone.conf cleanup "$sourceC" $statsC $loglevelC $dryrunC $extraC
else:
  !rclone --config=/root/.config/rclone/rclone.conf copy "$sourceC" "$destinationC" $transfersC $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $dryrunC $extraC

##### Log Output
if Output_Log_File != "OFF" and Mode != "Config":

    ##### Rename log file and output settings.
    !mv /root/.rclone_log/rclone_log.txt /root/.rclone_log/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).txt
    with open("/root/.rclone_log/" + Mode + "_settings.txt", "w") as f:
        f.write("Mode: " + Mode + \
            "\nCompare: " + Compare + \
            "\nSource: \"" + Source + \
            "\"\nDestination: \"" + Destination + \
            "\"\nTransfers: " + str(Transfers) + \
            "\nCheckers: " + str(Checkers) + \
            "\nSkip files that are newer on the destination: " + str(Skip_files_that_are_newer_on_the_destination) + \
            "\nSkip all files that exist: " + str(Skip_all_files_that_exist) + \
            "\nDo not cross filesystem boundaries: " + str(Do_not_cross_filesystem_boundaries) + \
            "\nDo not update modtime if files are identical: " + str(Do_not_update_modtime_if_files_are_identical) + \
            "\nDry-Run: " + str(Dry_Run) + \
            "\nOutput Log Level: " + Output_Log_File + \
            "\nExtra Arguments: \"" + Extra_Arguments + \
            "\"\nSync Moden: " + Sync_Mode + \
            "\nTrack Renames: " + str(Track_Renames) + \
            "\nDeduplicate Mode: " + Deduplicate_Mode + \
            "\nDeduplicate Use Trash: " + str(Deduplicate_Use_Trash))

    ##### Compressing log file.
    !rm -f /root/rclone_log.zip
    !zip -r -q -j -9 /root/rclone_log.zip /root/.rclone_log/
    !rm -rf /root/.rclone_log/
    !mkdir -p -m 666 /root/.rclone_log/

    ##### Send Log
    if os.path.isfile("/root/rclone_log.zip") == True:
        try:
            files.download("/root/rclone_log.zip")
            !rm -f /root/rclone_log.zip
            display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">Sending log to your browser...</h2><br></center>"))
        except:
            !mv /root/rclone_log.zip /content/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).zip
            display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">You can use file explorer to download the log file.</h2><br><img src=\"https://minormole.github.io/RcloneLab/res/rclonelab/01.png\"><br></center>"))
    else:
        clear_output()
        display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">There is no log file.</h2><br></center>"))


### Operation has been successfully completed.
if Mode != "Config":
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">✅ Operation has been successfully completed.</h2><br></center>"))


##### Automatically clear terminal output if the checkbox's value on the top is set to True.
if automatically_clear_cell_output is True:
    clear_output()
else:
	pass##### Automatically clear terminal output if the checkbox's value on the top is set to True.
if automatically_clear_cell_output is True:
    clear_output()
else:
	pass

In [ ]:
#@title 1.安装依赖(2min左右，中途有个输入国家代码)
!sudo apt install build-essential
!sudo apt install libcurl4-openssl-dev
!sudo apt install libsqlite3-dev
!sudo apt install pkg-config
!sudo apt install git
!sudo apt install curl
!sudo apt install libnotify-dev
!echo 'deb http://download.opensuse.org/repositories/home:/npreining:/debian-ubuntu-onedrive/xUbuntu_20.04/ /' | sudo tee /etc/apt/sources.list.d/home:npreining:debian-ubuntu-onedrive.list
!curl -fsSL https://download.opensuse.org/repositories/home:npreining:debian-ubuntu-onedrive/xUbuntu_20.04/Release.key | gpg --dearmor | sudo tee /etc/apt/trusted.gpg.d/home_npreining_debian-ubuntu-onedrive.gpg > /dev/null
!sudo apt update
!sudo apt install onedrive

In [ ]:
#@title 2.配置文件
!mkdir -p  /root/.config/onedrive
!mkdir -p  /content/sample_data/GDrive_sync
#@markdown 配置文件详细https://www.chuxin911.com/digital_management_onedrive_client_for_linux_20220227/
#@markdown ### 同步方向：content/sample_data/GDrive_sync ----> GDrive_sync
#@markdown ### 下面有个upload_only = "true"改成"false"就是双向同步
config_content = '''# 设置同步文件夹位置
sync_dir = "/content/sample_data"

# 忽略所有隐藏文件和文件夹
skip_dotfiles = "true"
upload_only = "true"
# 是否单向上传
no_remote_delete = "true"
# 是否设定不删除云端文件,为了云端数据安全'''
sync_list = "GDrive_sync"
with open('/content/config', 'w') as f:
    f.write(config_content)
with open('/content/sync_list', 'w') as f:
    f.write(sync_list)
!cp /content/config /root/.config/onedrive/config
!cp /content/sync_list /root/.config/onedrive/sync_list


In [ ]:
#@title 3.开始同步
!onedrive --synchronize --resync
#@markdown 更新配置文件后加上参数--resync

#@markdown 需要连接OneDrive账户,按指引复制最后的链接回车

# （资源分流）临时上传文件（包括批量上传）
* 1.bayfiles 没有说明过期时间，基本没限制
* 2.gofile 取决于服务器，无活跃下载十天之内删除文件
* 文件名有;/\.什么奇怪字符可能上传失败

In [ ]:
#@title 上传文件到https://bayfiles.com/
#@markdown 选择上传载点
url = "https://api.rapidshare.nu/upload" #@param ["https://api.openload.cc/upload", "https://api.bayfiles.com/upload", "https://api.lolabits.se/upload", "https://api.vshare.is/upload", "https://api.rapidshare.nu/upload"] {allow-input: true}
import requests
import os
import humanize
#@markdown 上传文件路径
file_path = "/content/sample_data/a.txt"   #@param {type:"string"}
file_name = os.path.basename(file_path)  # 文件名
file_size = os.path.getsize(file_path)  # 文件大小
response = requests.get(url)

# 开始上传
with open(file_path, 'rb') as f:
    files = {'file': (file_name, f)}
    response = requests.post(url, files=files)

# 输出结果
url_short = response.json()['data']['file']['url']['short']
print(f"{url_short}, {humanize.naturalsize(file_size)}, filename={file_name}")

In [ ]:
#@title 上传文件夹到https://bayfiles.com/
import os
import humanize
#@markdown 选择上传载点
url = "https://api.vshare.is/upload" #@param ["https://api.openload.cc/upload", "https://api.bayfiles.com/upload", "https://api.lolabits.se/upload", "https://api.vshare.is/upload", "https://api.rapidshare.nu/upload"] {allow-input: true}
folder_path = '/content/sample_data' #@param {type:"string"}

# 遍历文件夹中的所有文件
for file_name in os.listdir(folder_path):
    # 构造文件路径
    file_path = os.path.join(folder_path, file_name)

    # 如果当前路径是文件而不是文件夹，则下载文件
    if os.path.isfile(file_path):
        # 使用 curl 命令下载文件
        os.system(f'curl -F "file=@{file_path}" {url} -o output.json')
        with open('output.json', 'r') as f:
            s = f.read()

        with open('results.txt', 'a') as f:
            f.write(s + '\n')

import json

# 读取文件内容
with open('results.txt', 'r') as f:
    content = f.readlines()

# 存储所有 JSON 对象
json_objects = []

# 逐行解析 JSON 对象
current_json_object = ''
for line in content:
    if line.strip():  # 如果不是空行，将当前行添加到当前 JSON 对象
        current_json_object += line
    else:  # 如果是空行，表示当前 JSON 对象已经解析完毕
        json_objects.append(current_json_object)
        current_json_object = ''

# 解析每个 JSON 对象
for json_object in json_objects:
    data = json.loads(json_object)

    # 从 Python 字典中提取需要的值
    url_short = data['data']['file']['url']['short']
    size_bytes = data['data']['file']['metadata']['size']['bytes']
    filename = data['data']['file']['metadata']['name']
    print(f'{url_short}, {humanize.naturalsize(size_bytes)}, {filename}')
    # print("----")
    # print(f'{url_short}')


In [ ]:
#@title 上传文件到https://gofile.io/
import requests
import json
import time
import os
start_time = time.time()

file_path = "/content/sample_data/ansc.json" #@param {type:"string"}
file_size = os.path.getsize(file_path)
print("文件大小："+str(file_size))
response = requests.get('https://api.gofile.io/getServer')
data = json.loads(response.text)
server = data['data']['server']
print("当前最佳服务器："+server)

os.system(f'curl -F "file=@{file_path}" https://{server}.gofile.io/uploadFile -o output.json')
!curl -F file=@{file_path} https://{server}.gofile.io/uploadFile -o output.json
with open('output.json', 'r') as f:
    s = f.read()

data = json.loads(s)
url = data['data']['downloadPage']
print(url)


end_time = time.time()  # 记录结束时间
upload_time = end_time - start_time  # 计算上传时间
upload_speed = file_size / upload_time  # 计算上传速度
print("上传时间：{:.2f}秒".format(upload_time))
print("上传速度：{:.2f}MB/s".format(upload_speed / 1024 / 1024))


In [ ]:
#@title 上传文件夹到https://gofile.io/
import json
import os

response = requests.get('https://api.gofile.io/getServer')
data = json.loads(response.text)
server = data['data']['server']
print("当前最佳服务器："+server)
folder_path = '/content/sample_data/GDrive_syn' #@param {type:"string"}

# 删除已存在的输出文件
if os.path.exists('results_Gofile.txt'):
    os.remove('results_Gofile.txt')
flag = 0
#@markdown 注册gofile有自己的token，详细看那边说明
token = "m2obcFWuuiXcTnMwgl7BkoIK3FsMKkAC" #@param {type:"string"}
# 遍历文件夹中的所有文件
for file_name in os.listdir(folder_path):
    # 构造文件路径
    file_path = os.path.join(folder_path, file_name)
    print(file_path)
    # 如果当前路径是文件而不是文件夹，则下载文件
    if os.path.isfile(file_path):
        # 使用 curl 命令下载文件
        if flag == 0:
          os.system(f'curl -F token={token} -F "file=@{file_path}" https://{server}.gofile.io/uploadFile -o output.json')
          with open('output.json', 'r') as f:
              s = f.read()
          data = json.loads(s)
          folderId = data['data']['parentFolder']
          print(folderId)
          flag = 1
        else:
          print("i")
          os.system(f'curl -F "file=@{file_path}" -F token={token} -F folderId={folderId} https://{server}.gofile.io/uploadFile -o output.json')
          with open('output.json', 'r') as f:
              s = f.read()
        with open('results_Gofile.txt', 'a') as f:
            f.write(s + '\n')

# 读取文件内容
with open('results_Gofile.txt', 'r') as f:
    for line in f:
        # 解析当前行的 JSON 对象
        data = json.loads(line.strip())

        # 在这里处理解析出的数据
        # 例如提取某个字段的值并打印
        url_short = data['data']['downloadPage']
        file_name = data['data']['fileName']

        print(f'url_short={url_short}, file_name={file_name}')



# <img class="logo-img" width="32" height="32"  src="https://mega.io/wp-content/themes/megapages/megalib/images/megaicon.svg" alt="MEGA Logo" >（Mega）支持文件、文件夹下载
From https://colab.research.google.com/github/shirooo39/MiXLab/blob/master/MiXLab.ipynb#scrollTo=3GKtYuBbUP-c

In [ ]:
#@title 1.直接下载
# ============================= FORM ============================= #
URL = "https://mega.nz/folder/9n5UTJSB#Hi50VqAs7V1r2m1tENZBJA" #@param {type:"string"}
OUTPUT_PATH = "" #@param {type:"string"}
# @markdown > URL: Mega分享链接
# @markdown > OUTPUT_PATH：保存路径
# ================================================================ #

import sys, os, urllib.request
import time
import subprocess
import contextlib
from IPython.display import clear_output

HOME = os.path.expanduser("~")
if not os.path.exists(f"{HOME}/.ipython/mixlab.py"):
    hCode = "https://raw.githubusercontent.com/shirooo39/" \
                "MiXLab/master/resources/mixlab.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/mixlab.py")

from mixlab import (
    runSh,
    loadingAn,
)

if not OUTPUT_PATH:
  os.makedirs("downloads", exist_ok=True)
  OUTPUT_PATH = "downloads"
# Installing MEGAcmd
if not os.path.exists("/usr/bin/mega-cmd"):
    loadingAn()
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
    clear_output()

# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out

def transfare():
    import codecs
    decoder = codecs.getincrementaldecoder("UTF-8")()
    cmd = ["mega-get", URL, OUTPUT_PATH]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        print(line)

transfare()

In [ ]:
#@title 2.登录Mega（需要上传文件时）
# ============================= FORM ============================= #
# ================================================================ #

from functools import wraps
import errno
import os
import signal
import subprocess
import shlex

class TimeoutError(Exception):
    pass


def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator

if not os.path.exists("/root/.ipython/mixlab.py"):
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/shirooo39/MiXLab/master/resources/mixlab.py \
                    -O /root/.ipython/mixlab.py"
    run(split(shellCmd))
from mixlab import runSh

@timeout(10)
def runShT(args):
    return runSh(args, output=True)

# Installing MEGAcmd
if not os.path.exists("/usr/bin/mega-cmd"):
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
else:
    !pkill mega-cmd

# Enter MEGA credential
USERNAME = ""  # @param {type:"string"}
PASSWORD = ""  # @param {type:"string"}
if not (USERNAME == "" or PASSWORD == ""):
    try:
        runShT(f"mega-login {USERNAME} {PASSWORD}")
    except TimeoutError:
        runSh('mega-whoami', output=True)
else:
    print("Please enter your MEGA credential.")

In [ ]:
#@title 2.5.压缩文件夹（可选）
import os, zipfile

def make_zip(source_dir, output_filename):
    zipf = zipfile.ZipFile(output_filename, 'w')
    pre_len = len(os.path.dirname(source_dir))
    for parent, dirnames, filenames in os.walk(source_dir):
        for filename in filenames:
            pathfile = os.path.join(parent, filename)
            arcname = pathfile[pre_len:].strip(os.path.sep)     #相对路径
            zipf.write(pathfile, arcname)
    zipf.close()
#@markdown  指定要压缩的文件夹，左边复制路径
dir = "/content/sample_data" #@param {type:"string"}
zipFile = f"{dir}.zip"
make_zip(dir,zipFile)
print(f"已压缩,{zipFile}")

In [ ]:
#@title 3.上传
# ============================= FORM ============================= #
# Simple_torrent = False  # @param{type: "boolean"}
# Peerflix = False  # @param{type: "boolean"}
PATH_TO_FILE = "" # @param {type:"string"}
# @markdown > PATH_TO_FILE：需要上传的文件路径
# ================================================================ #

import time
import subprocess
import contextlib
from IPython.display import clear_output

# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out

def transfare():
    cmd = ""
    if Simple_torrent:
        cmd = ['mega-put', 'downloads', '/colab']
    elif Peerflix:
        cmd = ['mega-put', 'peerflix', '/colab']
    else:
        cmd = ['mega-put', PATH_TO_FILE, '/colab']
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        clear_output(wait=True)
        print(line)

try:
    transfare()
except FileNotFoundError:
    print("Please log into your MEGA account first!")

# kemono (mega链接在数组里面，因为有ip流量限制可能会失败)

In [ ]:
#@title 安装依赖
import os
from IPython.display import clear_output
!apt-get -y install -qq aria2
if not os.path.exists("/usr/bin/mega-cmd"):
    print("Installing MEGA ...")
    # !apt-get -y update
    !apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https
    !curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb
    !dpkg -i /var/cache/apt/archives/MEGAcmd.deb
    clear_output()
    print("MEGA is installed.")

In [ ]:
#@title 爬取页面
import requests
from bs4 import BeautifulSoup
import subprocess,sys,re
import threading
import time
from urllib.parse import urlsplit, parse_qs, unquote
import os
from concurrent.futures import ThreadPoolExecutor

savepath = "/content/0sss"
# savepath = os.path.join(savepath, "")
mega_links = {}
post_ids = []
skip_id = []
url = 'https://kemono.party/patreon/user/43005054'
user_id = re.search(r'\d+', url).group()
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
name = soup.find('span', {'itemprop': 'name'}).text.strip()

def download_link(link, folder_path):
    print("asdasd")
    parsed_link = urlsplit(link)
    max_attempts = 20
    attempt = 0
    if "mega" in link:
        mega_links[link] = folder_path
        os.makedirs(folder_path, exist_ok=True)
        # time.sleep(0.1)
        # !mega-get {link} "{folder_path}"  # 使用 megadl 下载链接
    elif "drive.google" in link:
        file_id = link.split("/")[5]
        print(f"id========================{file_id}")
        if "/folders/" in link:
          !gdown --fuzzy "{file_id}" -O "{folder_path}" --folder
        else:
          !gdown --fuzzy "{file_id}" -O "{folder_path}"  # 使用 gdown 下载链接
    elif "dropbox" in link:
        url = parsed_link.scheme + "://" + parsed_link.netloc + parsed_link.path
        if "/sh/" in link:
          !wget -P "{folder_path}" -O "archive.zip" -c {url}
        else:
          !wget -P "{folder_path}" -c {url}
    else:
        while attempt < max_attempts:
            query_string = parsed_link.query
            params = parse_qs(query_string)
            file_param = params['f'][0]
            file_name = os.path.basename(file_param)
            if os.path.isfile(os.path.join(folder_path, file_name)):
                break
            !aria2c --console-log-level=error -x 2 -s 2 --summary-interval 5 -d "{folder_path}" -o "{file_name}"  {link}
            if os.path.isfile(os.path.join(folder_path, file_name)):
                break
            else:
                attempt += 1
                print(f"Attempt {attempt} to download {link} failed, retrying...")
        else:
            print(f"Failed to download {link} after {max_attempts} attempts.")
# 处理单页的投稿，推送到队列
def process_page(url, queue):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    name = soup.find('span', {'itemprop': 'name'}).text.strip()
    articles = soup.find_all('article', {'class': 'post-card'})
    # 获取投稿链接，推送到队列
    for article in articles:
        post_id = re.search(r'\d+$', article.find('a')['href']).group()
        post_card_text = article.find('header', {'class': 'post-card__header'}).text.strip()
        link = article.find('a')['href']
        queue.put((link, post_id, post_card_text))

from multiprocessing import Pool

def process_post(queue):
    while True:
        item = queue.get()
        if item is None:
            break
        link, post_id, post_card_text = item
        i = 0
        response = requests.get('https://kemono.party' + link)
        soup = BeautifulSoup(response.text, 'html.parser')
        post_content = soup.find('div', {'class': 'post__body'})
        links = []
        for link in post_content.find_all(["a", "source"]):
            href = link.get("href") or link.get("src")
            if href.startswith("http") and "fanbox.cc" not in href:
              if href.endswith((".mp4", ".webm", ".wav", ".m4p", ".ogg", ".pdf", ".wav", ".ogg", ".jpg", ".jpeg", ".png", ".gif", ".zip", ".rar", ".7z")) or any(x in href for x in ["mega", "dropbox", "gofile", "drive"]):
                  links.append(href)
              else:
                  print(f"Skipped link {href} for post {post_id} of link: {href}")
        # for p in post_content.find_all('p'):
        #   links.append(p.text)
        links_set = set(links)  # 转换为集合去重
        links = list(links_set)  # 转换为列表
        post_card_text = post_card_text.replace('/', '_')
        # 将双引号替换为中文的引号
        post_card_text = post_card_text.replace('"', '“')
        print(f"{len(links)}::{links}")
        with ThreadPoolExecutor(max_workers=8) as executor:
          for link in links:
              executor.submit(download_link, link, f"{savepath}/{name}-{user_id}/{post_id}-{post_card_text}/")
        queue.task_done()

import queue
# 创建队列
q = queue.Queue()
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
try:
    text = soup.find('small').text.strip()
    result = re.search(r'\d+$', text)
    pages = int(result.group()) // 50 + 1
    for page in range(pages):
      page_url = f"{url}?o={page*50}"
      process_page(page_url,q)
except AttributeError:
    process_page(url,q)

# process_page(url,q)

# 创建线程池并下载链接
num_threads = 8
threads = []
for i in range(num_threads):
    t = threading.Thread(target=process_post, args=(q,))
    threads.append(t)
    t.start()

# 等待所有任务完成
q.join()

# 停止所有线程
for i in range(num_threads):
    q.put(None)
for t in threads:
    t.join()

In [ ]:
#@title mega
mega_links = {"https://mega.nz/#!4TZUWI6B!_Qxa8FNGmYTh9D4asOEIqEKBI6FWqEidnFrSRAFjaTw":"/content/221111"}
print(mega_links)
import subprocess
import contextlib,os
newlines = ['\n', '\r\n', '\r']
def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out

for link, folder_path in mega_links.items():
    os.makedirs(folder_path, exist_ok=True)
    print(link)
    print(folder_path)
    command = ["mega-get", link, folder_path]
    proc = subprocess.Popen(command,stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,)
    print(proc)
    counter = 0
    for line in unbuffered(proc):
        counter += 1
        if counter % 20 == 0:
            print(line)

# 其他工具
From https://colab.research.google.com/github/shirooo39/MiXLab/blob/master/MiXLab.ipynb#scrollTo=3GKtYuBbUP-c

In [ ]:
#@title <img src="https://img.icons8.com/color/100/000000/youtube-play.png" width="32" height="32"  align="left" alt=""/>YouTube  视频下载</font>
# ============================= FORM ============================= #
# @markdown <font size=5>直接运行，会出现下载界面</font>
Archive = False
# ================================================================ #

import os, uuid, urllib.parse
import ipywidgets as widgets

from glob import glob
from urllib.parse import urlparse, parse_qs
from IPython.display import HTML, clear_output, YouTubeVideo
from IPython.utils.io import ask_yes_no
from google.colab import output, files

Links = widgets.Textarea(placeholder='''Video/Playlist Link
(one link per line)''')

VideoQ = widgets.Dropdown(options=["Best Quality (VP9 upto 4K)", "Best Compatibility (H.264 upto 1080p)"])

AudioQ = widgets.Dropdown(options=["Best Quality (Opus)", "Best Compatibility (M4A)"])

Subtitle = widgets.ToggleButton(value=True, description="Subtitle", button_style="info", tooltip="Subtitle")

SavePathYT = widgets.Dropdown(options=["/content", "/content/downloads"])

AudioOnly = widgets.ToggleButton(value=False, description="Audio Only", button_style="", tooltip="Audio Only")

Resolution = widgets.Select(options=["Highest", "4K", "1440p", "1080p", "720p", "480p", "360p", "240p", "144p"], value="Highest")

Extension = widgets.Select(options=["mkv", "webm"], value="mkv")

UsernameYT = widgets.Text(placeholder="Username")

PasswordYT = widgets.Text(placeholder="Password")

SecAuth = widgets.Text(placeholder="2nd Factor Authentication")

VideoPW = widgets.Text(placeholder="Video Password")

GEOBypass = widgets.Dropdown(options=["Disable", "Hide", "AD", "AE", "AF", "AG", "AI", "AL", "AM", "AO", "AQ", "AR", "AS", "AT", "AU", "AW", "AX", "AZ", "BA", "BB", "BD", "BE", "BF", "BG", "BH", "BI", "BJ", "BL", "BM", "BN", "BO", "BQ", "BR", "BS", "BT", "BV", "BW", "BY", "BZ", "CA", "CC", "CD", "CF", "CG", "CH", "CI", "CK", "CL", "CM", "CN", "CO", "CR", "CU", "CV", "CW", "CX", "CY", "CZ", "DE", "DJ", "DK", "DM", "DO", "DZ", "EC", "EE", "EG", "EH", "ER", "ES", "ET", "FI", "FJ", "FK", "FM", "FO", "FR", "GA", "GB", "GD", "GE", "GF", "GG", "GH", "GI", "GL", "GM", "GN", "GP", "GQ", "GR", "GS", "GT", "GU", "GW", "GY", "HK", "HM", "HN", "HR", "HT", "HU", "ID", "IE", "IL", "IM", "IN", "IO", "IQ", "IR", "IS", "IT", "JE", "JM", "JO", "JP", "KE", "KG", "KH", "KI", "KM", "KN", "KP", "KR", "KW", "KY", "KZ", "LA", "LB", "LC", "LI", "LK", "LR", "LS", "LT", "LU", "LV", "LY", "MA", "MC", "MD", "ME", "MF", "MG", "MH", "MK", "ML", "MM", "MN", "MO", "MP", "MQ", "MR", "MS", "MT", "MU", "MV", "MW", "MX", "MY", "MZ", "NA", "NC", "NE", "NF", "NG", "NI", "NL", "NO", "NP", "NR", "NU", "NZ", "OM", "PA", "PE", "PF", "PG", "PH", "PK", "PL", "PM", "PN", "PR", "PS", "PT", "PW", "PY", "QA", "RE", "RO", "RS", "RU", "RW", "SA", "SB", "SC", "SD", "SE", "SG", "SH", "SI", "SJ", "SK", "SL", "SM", "SN", "SO", "SR", "SS", "ST", "SV", "SX", "SY", "SZ", "TC", "TD", "TF", "TG", "TH", "TJ", "TK", "TL", "TM", "TN", "TO", "TR", "TT", "TV", "TW", "TZ", "UA", "UG", "UM", "US", "UY", "UZ", "VA", "VC", "VE", "VG", "VI", "VN", "VU", "WF", "WS", "YE", "YT", "ZA", "ZM", "ZW"])

ProxyYT = widgets.Text(placeholder="Proxy URL")

MinSleep = widgets.BoundedIntText(value=0, min=0, max=300, step=1, description="Min:")

MaxSleep = widgets.BoundedIntText(value=0, min=0, max=300, step=1, description="Max:")

ExtraArg = widgets.Text(placeholder="Extra Arguments")

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html

def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def upload_archive():
  if ask_yes_no("Do you already have an archive file? (y/n)", default="", interrupt=""):
    try:
      display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Please upload an archive from your computer.</h2><br>"))
      UploadConfig = files.upload().keys()
      clear_output(wait=True)
      if len(UploadConfig) == 0:
        return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been cancelled during upload file.</h2><br></center>"))
      elif len(UploadConfig) == 1:
        for fn in UploadConfig:
          if os.path.isfile("/content/" + fn):
            get_ipython().system_raw("mv -f " + "\"" + fn + "\" /root/.youtube-dl.txt && chmod 666 /root/.youtube-dl.txt")
            AudioOnly.observe(AudioOnlyChange)
            Subtitle.observe(SubtitleChange)
            AudioQ.observe(AudioQChange)
            ShowYT()
          else:
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been failed during upload file.</h2><br></center>"))
      else:
        for fn in UploadConfig:
          get_ipython().system_raw("rm -f " + "\"" + fn + "\"")
        return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Please uploading only one file at a time.</h2><br></center>"))
    except:
      clear_output(wait=True)
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Error occurred during upload file.</h2><br></center>"))
  else:
    get_ipython().system_raw("touch '/root/.youtube-dl.txt'")
    AudioOnly.observe(AudioOnlyChange)
    Subtitle.observe(SubtitleChange)
    AudioQ.observe(AudioQChange)
    ShowYT()

def RefreshPathYT():
  if os.path.exists("/content/drive/"):
    if os.path.exists("/content/drive/Shared drives/"):
      SavePathYT.options = ["/content", "/content/downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/") + glob("/content/drive/Shared drives/*/")
    else:
      SavePathYT.options = ["/content", "/content/downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/")
  else:
    SavePathYT.options = ["/content", "/content/downloads"]

def AudioOnlyChange(change):
  if change["type"] == "change" and change["new"]:
    VideoQ.disabled = True
    Subtitle.disabled = True
    if Subtitle.value:
      Subtitle.button_style = "info"
    else:
      Subtitle.button_style = ""
    Resolution.disabled = True
    Extension.options = ["best", "aac", "flac", "mp3", "m4a", "opus", "vorbis", "wav"]
    Extension.value = "best"
    AudioOnly.button_style = "info"
  elif change["type"] == "change" and change["new"] == False:
    VideoQ.disabled = False
    Subtitle.disabled = False
    if Subtitle.value:
      Subtitle.button_style = "info"
    else:
      Subtitle.button_style = ""
    Resolution.disabled = False
    if AudioQ.value == "Best Quality (Opus)":
      Extension.options = ["mkv", "webm"]
    else:
      Extension.options = ["mkv", "mp4", "webm"]
    Extension.value = "mkv"
    AudioOnly.button_style = ""

def SubtitleChange(change):
  if change["type"] == "change" and change["new"]:
    Subtitle.button_style = "info"
  elif change["type"] == "change" and change["new"] == False:
    Subtitle.button_style = ""

def AudioQChange(change):
  if change["type"] == "change" and change["new"] == "Best Quality (Opus)":
    Extension.options = ["mkv", "webm"]
    Extension.value = "mkv"
  elif change["type"] == "change" and change["new"] == "Best Compatibility (M4A)":
    Extension.options = ["mkv", "mp4", "webm"]
    Extension.value = "mkv"

def ShowYT():
  clear_output(wait=True)
  RefreshPathYT()
  display(widgets.HBox([widgets.VBox([widgets.HTML("<b style=\"color:#888888;\">Link:</b>"), Links,
                                      widgets.HTML("<b style=\"color:#888888;\">For website that require an account:</b>"), UsernameYT, PasswordYT, SecAuth, VideoPW,
                                      widgets.HTML("<b><a href=\"https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2#Officially_assigned_code_elements\" target=\"_blank\">GEO Bypass Country:</a></b>"), GEOBypass,
                                      widgets.HTML("<b style=\"color:#888888;\">Proxy:</b>"), ProxyYT,
                                      widgets.HTML("<b style=\"color:#888888;\">Sleep Interval (second):</b>"), MinSleep, MaxSleep]),
                        widgets.VBox([widgets.HTML("<b style=\"color:#888888;\">Video Quality:</b>"), VideoQ, widgets.HTML("<b style=\"color:#888888;\">Resolution:</b>"), Resolution,
                                      widgets.HTML("<b style=\"color:#888888;\">Audio Quality:</b>"), AudioQ, widgets.HTML("<b style=\"color:#888888;\">Extension:</b>"), Extension,
                                      widgets.HTML("<b style=\"color:#888888;\">Extra Options:</b>"), widgets.HBox([Subtitle, AudioOnly]),
                                      widgets.HTML("<b style=\"color:#888888;\">Extra Arguments:</b>"), ExtraArg])]), HTML("<h4 style=\"color:#888888;\">Save Location:</h4>"),
          SavePathYT, MakeButton("Refresh", RefreshPathYT, ""))
  if not os.path.exists("/content/drive/"):
#    display(HTML("*If you want to save in Google Drive please run the cell below."))
    display(HTML("<br>"), MakeButton("Download", DownloadYT, "info"))

def DownloadYT():
  if Links.value.strip():
    Count = 0
    Total = str(len(Links.value.splitlines()))
    # Account Check
    if UsernameYT.value.strip() and PasswordYT.value.strip():
      accountC = "--username \"" + UsernameYT.value + "\" --password \"" + PasswordYT.value + "\""
    else:
      accountC = ""
    if SecAuth.value.strip():
      secauthC = "-2 " + SecAuth.value
    else:
      secauthC = ""
    if VideoPW.value.strip():
      videopwC = "--video-password " + VideoPW.value
    else:
      videopwC = ""
    # Proxy
    if ProxyYT.value.strip():
      proxyytC = "--proxy " + ProxyYT.value
    else:
      proxyytC = ""
    # GEO Bypass
    if GEOBypass.value == "Disable":
      geobypass = ""
    elif GEOBypass.value == "Hide":
      geobypass = "--geo-bypass"
    else:
      geobypass = "--geo-bypass-country " + GEOBypass.value
    # Video Quality
    if VideoQ.value == "Best Quality (VP9 upto 4K)":
      videoqC = "webm"
    else:
      videoqC = "mp4"
    # Audio Quality
    if AudioQ.value == "Best Quality (Opus)":
      audioqC = "webm"
    else:
      audioqC = "m4a"
    # Audio Only Check
    if AudioOnly.value:
      subtitleC = ""
      thumbnailC = ""
      extC = "-x --audio-quality 0 --audio-format " + Extension.value
      codecC = "bestaudio[ext=" + audioqC + "]/bestaudio/best"
    else:
      if Subtitle.value:
        subtitleC = "--all-subs --convert-subs srt --embed-subs"
      else:
        subtitleC = ""
      if Extension.value == "mp4":
        thumbnailC = "--embed-thumbnail"
      else:
        thumbnailC = ""
      extC = "--merge-output-format " + Extension.value
      if Resolution.value == "Highest":
        codecC = "bestvideo[ext=" + videoqC + "]+bestaudio[ext=" + audioqC + "]/bestvideo+bestaudio/best"
      else:
        codecC = "bestvideo[ext=" + videoqC + ",height<=" + Resolution.value.replace("4K", "2160").replace("p", "") + "]+bestaudio[ext=" + audioqC + "]/bestvideo[height<=" + Resolution.value.replace("4K", "2160").replace("p", "") + "]+bestaudio/bestvideo+bestaudio/best"
    # Archive
    if os.path.isfile("/root/.youtube-dl.txt"):
      archiveC = "--download-archive \"/root/.youtube-dl.txt\""
    else:
      archiveC = ""
    # Sleep Interval
    if MinSleep.value > 0 and MaxSleep.value > 0:
      minsleepC = "--min-sleep-interval " + MinSleep.value
      maxsleepC = "--max-sleep-interval " + MaxSleep.value
    else:
      minsleepC = ""
      maxsleepC = ""
    # Extra Arguments
    extraargC = ExtraArg.value
    for Link in Links.value.splitlines():
      clear_output(wait=True)
      Count += 1
      display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Processing link " + str(Count) + " out of " + Total + "</h3>"))
      if "youtube.com" in Link or "youtu.be" in Link:
        display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Currently downloading...</h3><br>"), YouTubeVideo(Link, width=640, height=360), HTML("<br>"))
      else:
        display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Currently downloading <a href=\"" + Link + "\">" + Link + "</a></h3><br>"))
      if ("youtube.com" in Link or "youtu.be" in Link) and "list=" in Link:
        !youtube-dl -i --no-warnings --yes-playlist --add-metadata $accountC $secauthC $videopwC $minsleepC $maxsleepC $geobypass $proxyytC $extC $thumbnailC $subtitleC $archiveC $extraargC -f "$codecC" -o "/root/.YouTube-DL/%(playlist)s/%(playlist_index)s - %(title)s.%(ext)s" "$Link"
      else:
        !youtube-dl -i --no-warnings --yes-playlist --add-metadata $accountC $secauthC $videopwC $minsleepC $maxsleepC $geobypass $proxyytC $extC $thumbnailC $subtitleC $archiveC $extraargC -f "$codecC" -o "/root/.YouTube-DL/%(title)s.%(ext)s" "$Link"
      if not os.path.exists(SavePathYT.value):
        get_ipython().system_raw("mkdir -p -m 666 " + SavePathYT.value)
      get_ipython().system_raw("mv /root/.YouTube-DL/* '" + SavePathYT.value + "/'")
    # Archive Download
    if os.path.isfile("/root/.youtube-dl.txt"):
      files.download("/root/.youtube-dl.txt")
    ShowYT()

if not os.path.isfile("/usr/local/bin/youtube-dl"):
  get_ipython().system_raw("rm -rf /content/sample_data/ && mkdir -p -m 666 /root/.YouTube-DL/ && apt-get install atomicparsley && curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl && chmod a+rx /usr/local/bin/youtube-dl")
if Archive:
  upload_archive()
else:
  AudioOnly.observe(AudioOnlyChange)
  Subtitle.observe(SubtitleChange)
  AudioQ.observe(AudioQChange)
  ShowYT()

### [iwara视频下载](https://github.com/hare1039/iwara-dl)


In [ ]:
# ==============================#============================== #
# ==============================#============================== #
#@title 1.安装依赖
import glob
import pathlib
import shutil
import subprocess
from IPython.display import clear_output
from google.colab import files


iwara_download_list = []


class IwaraDL:
    def __init__(self):
        self.main_path = pathlib.Path('/content/tools/iwara-dl')
        self.download_path = self.main_path.joinpath('Downloads')
        self.text_file_name = 'download.txt'
        self.text_file_path = self.main_path.joinpath(self.text_file_name)

        return None


    def execute(self, command, show_output=False, cwd=None):
        try:
            with subprocess.Popen(command, shell=True, cwd=cwd,
                stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                bufsize=1, universal_newlines=True
            ) as process:
                if show_output is True:
                    for line in process.stdout:
                        print(line, end='')
                else:
                    pass
        except Exception as unknown_error:
            print(f'[Error]: {unknown_error}')

            return exit
        else:
            return None


    def install(self):
        # clone_path = str(self.main_path)
        install_command = [
            'apt -y install jq',
            'apt -y install python3-bs4',
            f'git clone https://github.com/hare1039/iwara-dl.git {self.main_path}'
        ]

        try:
            for command in install_command:
                self.execute(command, show_output=False)
                subprocess.run(command, shell=True)
        except Exception as unknown_error:
            print(f'[Error]: {unknown_error}')

            return exit
        else:
            self.download_path.mkdir(parents=True, exist_ok=True)

            return None


    def add_download_url(self):
        while True:
            clear_output()

            print('Type in a URL or "exit" to stop...\n')
            new_url = input('URL: ')

            if new_url == 'exit':
                clear_output()

                break
            elif new_url == '':
                continue
            else:
                iwara_download_list.append(new_url)

        return None


    def add_download_file(self):
        try:
            self.text_file_path.unlink()
        except:
            pass

        uploaded = files.upload()

        for filename in uploaded.keys():
            shutil.move(filename, self.text_file_path)

            clear_output()

            print(f'Moved "{filename}" into "{self.text_file_path}"')

        try:
            with open(self.text_file_path, 'r', encoding='UTF-8') as txt_file:
                global iwara_download_list

                file = txt_file.read()
                iwara_download_list.append(file)
        except Exception as unknown_error:
            print(f'[Error]: {unknown_error}')

            return exit
        finally:
            txt_file.close()


    def show_download_list(self):
        if not iwara_download_list:
            print('The download list is empty.')
        else:
            print(*iwara_download_list, sep='\n')

        return None


    def clear_download_list(self):
        iwara_download_list.clear()

        return None


    def purge_download_folder(self):
        files = glob.glob(f'{self.download_path}/*')

        try:
            for f in files:
                pathlib.Path(f).unlink()
        except Exception as unknown_error:
            print(f'[Error]: {unknown_error}')

            return exit


if __name__ == '__main__':
    IwaraDL().install()

In [ ]:
# ------------------------------#------------------------------ #
#@title 2.添加下载链接
#@markdown <font size='5'>Add downloads from URL or 'download.txt'</font>
command = 'add_download_from_url' #@param ['add_download_from_url', 'add_download_from_file', 'show_download_list', 'clear_download_list', 'purge_download_folder']
# ------------------------------#------------------------------ #
#@markdown 注意链接不要包含用户名，https://www.iwara.tv/video/zYiWkPQc7SbVza/gangbang 删掉后面/gangbang
from IPython.display import clear_output


def main():
    command_list = [
        'add_download_from_url', 'add_download_from_file',
        'show_download_list', 'clear_download_list',
        'purge_download_folder'
    ]

    if command == command_list[0]:
        IwaraDL().add_download_url()
    elif command == command_list[1]:
        IwaraDL().add_download_file()
    elif command == command_list[2]:
        IwaraDL().show_download_list()
    elif command == command_list[3]:
        IwaraDL().clear_download_list()
    elif command == command_list[4]:
        IwaraDL().purge_download_folder()
    else:
        pass

    return None


if __name__ == '__main__':
    try:
        main()
    except KeyboardInterrupt:
        clear_output()

        exit
    except NameError:
        print('[Error]: Please make sure you have already run the cell "[Install] ecchi.iwara-dl" first!')

        exit
    except Exception as unknown_error:
        print(f'[Error]: {unknown_error}')

        exit
    else:
        pass


In [ ]:
# ------------------------------#------------------------------ #
#@title 3.开始下载

#@markdown Downloads are stored in "/content/tools/iwara-dl/Downloads"

#@markdown ---

#@markdown <font size='5'><center>Options</center></font>
hide_progress = False #@param {type:"boolean"}
download_username = False #@param {type:"boolean"}
clear_list_on_finish = False #@param {type:"boolean"}

#@markdown ---

#@markdown > The option 'download_username'' treats the download list as username.<br>
#@markdown > Tick this option if you want to download a whole collection from a username.
#@markdown >
#@markdown > For example:<br>
#@markdown > To download everything (video only) from username "AutumnJelly", add the username --> autumnjelly <-- into the download list and tick this option. Only type in the username and not the full url: ecchi.iwara.tv/users/autumnjelly
#@markdown >
#@markdown > Do NOT mix the download list with URL and/or Username! That might break the downloader!
# ------------------------------#------------------------------ #

import pathlib
from IPython.display import clear_output


def main():
    clear_output()

    main_path = pathlib.Path('/content/tools/iwara-dl')
    dl_path = main_path.joinpath('Downloads')

    if not iwara_download_list:
        print('The download list is empty.')
    else:
        if hide_progress is True:
            n = '-n'
        else:
            n = ''

        if download_username is True:
            tc = '-t -c'

            for dir in iwara_download_list:
                dl_path.joinpath(dir).mkdir(parents=True, exist_ok=True)
        else:
            tc = ''

        for downloads in iwara_download_list:
            IwaraDL().execute(f'bash /content/tools/iwara-dl/iwara-dl.sh {tc} {n} {downloads}', show_output=True, cwd=dl_path)

        if clear_list_on_finish is True:
            iwara_download_list.clear()
        else:
            pass

    return None


if __name__ == '__main__':
    try:
        main()
    except KeyboardInterrupt:
        clear_output()
        exit
    except Exception as unknown_error:
        print(f'[Error]: {unknown_error}')
        exit
    else:
        pass